In [1]:
# Imports, as always...
import numpy as np
from fpylll import IntegerMatrix, LLL, GSO
from copy import deepcopy
import cirq

In [2]:
# Seed for random generation.
np.random.seed(42)

In [3]:
# The integer to be factored.
p, q = 659, 3
N = p * q

# What is the bit-length of the integer?
N_bit_length = N.bit_length()

# Convert to binary (drop the '0b' prefix).
N_binary = bin(N)[2:]

print(f'Integer to be factored: N = {N} (into the factors p = {p} and q = {q}).')
print(f'This has bit-length {N_bit_length}\n')

# The claimed lattice dimension to factor this integer is l * log N / log log N.
l = 1

n = np.log2(N)
m = (l * n) / np.log2(n)

# Round them, after the fact.
n, m = int(np.floor(n)), int(np.floor(m))

print(f'The claim is that we need only a lattice of dimension {m}.')

# Here are some primes that we can use.
primes = [2,3,5,7,11,13,17,19,23,29,31,37,41,43,47]

Integer to be factored: N = 1977 (into the factors p = 659 and q = 3).
This has bit-length 11

The claim is that we need only a lattice of dimension 3.


In [4]:
# Precision parameter.
c = 1.5

# Produce the random permutation for the diagonal.
f = np.random.permutation([(i + 1) // 2 for i in range(1, m + 1)])

# Create a zero matrix and add in the diagonal permutation.
B = np.zeros(shape=(m, m))
np.fill_diagonal(B, f)

# Create the extra final row and stick it on.
final_row = np.round(10 ** c * np.log(np.array(primes[:m])))
B = np.vstack((B, final_row))

# fpylll doesn't like numyp arrays, so convert it to a stnadard array.
B = [[int(b) for b in bs] for bs in B]

# Convert B to a matrix of integers (in fpylll's own type).
B = IntegerMatrix.from_matrix(B)
print(f'B = \n{B}')

# Define the target vector.
t = np.zeros(m + 1)
t[-1] = np.round(10 ** c * np.log(N))
t = tuple(t.astype(int))

# And again, if t could be a bunch of integers, that would be swell.
print(f'\nt = \n{t}\n')

# LLL-reduction hyperparameter (using 0.75, according to Wikipedia)
delta = .75

D = deepcopy(B).transpose()
LLL.reduction(D, delta)
print(f'D (transposed) = \n{D}\n')

M = GSO.Mat(D, update=True)
w = M.babai(t)

A = IntegerMatrix(2 * D.nrows, D.ncols)
for i in range(D.nrows):
    for j in range(D.ncols):
        A[i, j] = D[i, j]

b = np.array(t)
for i in reversed(range(D.nrows)):
    for j in range(D.ncols):
        A[i + D.nrows, j] = int(b[j])
    b -= w[i] * np.array(D[i])

print("Which way is each coefficient rounded?")
M = GSO.Mat(A, update=True)
rounding_direction = []
for i in range(D.nrows):
    mu = M.get_mu(i + D.nrows, i)
    print(f'mu={mu} c={w[i]}')
    rounding_direction.append(w[i] > mu)
  
b_op = np.array(D.multiply_left(w))
residual_vector = b_op - np.array(t)
print(f'\nb_op = \n{b_op}\n')
print(f'Hence, the residual vector is \n{residual_vector}\n')
print(f'This has a distance of {round(np.linalg.norm(residual_vector), 3)} to the target vector.')

B = 
[  1  0  0 ]
[  0  1  0 ]
[  0  0  2 ]
[ 22 35 51 ]

t = 
(0, 0, 0, 240)

D (transposed) = 
[  1 -2 2  3 ]
[ -4  1 2 -2 ]
[ -3  2 0  4 ]

Which way is each coefficient rounded?
mu=21.666666666666668 c=22
mu=-20.497409326424872 c=-20
mu=33.581188433428665 c=34

b_op = 
[  0   4   4 242]

Hence, the residual vector is 
[0 4 4 2]

This has a distance of 6.0 to the target vector.


In [5]:
# Reformatting after this step to make subsequent steps easier.
def integer_matrix_to_numpy(M):
  m, n = M.nrows, M.ncols
  D = np.zeros((m, n), dtype=int)
  M.to_matrix(D)
  return D

B = integer_matrix_to_numpy(B)
D = integer_matrix_to_numpy(D.transpose())
t = np.array(t)

In [6]:
# From the rounding directions, establish the signs for each operator.
step_signs = - (np.array(rounding_direction).astype(int) * 2 - 1)
print(f'Due to the roundings, our operators will have signs:\n{step_signs}')

Due to the roundings, our operators will have signs:
[-1 -1 -1]


In [7]:
# Define the circuit.
circuit = cirq.LineQubit.range(D.shape[0])

# Add the appropriate operator to each qubit.
operators = []
for i, sign in zip(circuit, step_signs):
    operator = sign * ((cirq.I(i) + -cirq.Z(i)) / 2)
    operators.append(operator)
    
# Define the Hamiltonian.
H = cirq.PauliSum()
for j in range(D.shape[0]):
    h = residual_vector[j] 
    for i in range(D.shape[1]):
        h += operators[i] * D[j, i]
    H += h ** 2
    
print(f'H = \n{H}')

H = 
43.5*I-4.0*Z(q(0))*Z(q(1))+2.5*Z(q(0))*Z(q(2))-3.5*Z(q(1))-4.0*Z(q(2))+3.0*Z(q(1))*Z(q(2))-1.5*Z(q(0))


In [8]:
# Pretty printing the Hamiltonian.
string_H = str(H)
string_H = string_H.replace('+', '\n+')
string_H = string_H.replace('-', '\n-')
string_H = string_H.replace('*', ' * ')
print(string_H)

43.500 * I
-4.000 * Z(q(0)) * Z(q(1))
+2.500 * Z(q(0)) * Z(q(2))
-3.500 * Z(q(1))
-4.000 * Z(q(2))
+3.000 * Z(q(1)) * Z(q(2))
-1.500 * Z(q(0))
